# A beginner's guide of using multiple ML algorithms to find out the best one, based on accuracy score.

The Algorithms used in this notebook are:
1. DecisionTreeClassifier and GridSearchCV
2. LogisticRegression
3. RandomForestClassfier

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        df = pd.read_csv(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

No null values in the dataset.

In [ ]:
df.quality.unique()

Classifying the data into: 
* 3,4,5 -> Bad (0)
* 6,7,8 -> Good (1)

In [ ]:
df['quality'].dtypes

In [ ]:
df[['quality']] = df['quality'].apply(lambda x : (x - x) if int(x) < 6 else ((x - x) + 1))

In [ ]:
df

In [ ]:
#Splitting y and X
y = df['quality']
X = df.drop('quality', axis = 1)

In [ ]:
y

In [ ]:
#Splitting into test and train data
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

**DecisionTreeClassifier and GridSearchCV**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
d_model = DecisionTreeClassifier()
criterion = ['gini', 'entropy']
max_depth = [1, 3, 5, None]
splitter = ['best', 'random']

In [ ]:
grid = GridSearchCV(estimator = d_model, cv= 3, param_grid=dict(criterion=criterion, max_depth=max_depth, splitter= splitter))

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print (grid.best_score_, grid.score(X_test, y_test))

**LogisticRegression**

In [ ]:
from sklearn.linear_model import LogisticRegression
l_model = LogisticRegression(max_iter=10000)
l_model.fit(X_train, y_train)

In [ ]:
l_model.score(X_test, y_test)

This model performed better than Decision Tree. Let's try RandomForrestClassifier now

**RandomForestClassfier**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

def get_score(n_estimators):
    my_pipeline = Pipeline(steps=[
        ('preprocessor', SimpleImputer()),
        ('model', RandomForestClassifier(n_estimators, random_state=42))
    ])
    scores = -1 * cross_val_score(my_pipeline, X_train, y_train,
                                  cv=3,
                                  scoring='neg_mean_absolute_error')
    return scores.mean()

In [ ]:
results = {}
for i in range(50,1000, 50):
    results[i] = get_score(i)

In [ ]:
plt.plot(list(results.keys()), list(results.values()))
plt.show()

From the above graph, we learn that for n_estimators = 400, the best score can be achieved

In [ ]:
r_forest = RandomForestClassifier(n_estimators=200)
r_forest.fit(X_train, y_train)
r_forest.score(X_test, y_test)

In [ ]:
y_valid = r_forest.predict(X_test)
y_valid

Calculatind MAE for RandomForestClassifier

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_valid, y_test)

In [ ]:
rfc_eval = cross_val_score(estimator = r_forest, X = X_train, y = y_train, cv = 10)
rfc_eval.mean()*100

Thank you for going through the  notebook. I am a beginner in data science, and would love to get some comments on my work above.